In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import functools
import json
from multiprocessing import Pool

# importing functions from project libraries
from SharePointFunctions import read_SPList
from OVERRIDE_FUNCTIONS import override_dataframe

# PROJECT METADATA

### Reading OneSource and OneSource Enrich Snowflake Tables

In [0]:
# Read recipe inputs
PDA_STG_PROJECT_REFERENCE_OS_PORTFOLIO = dataiku.Dataset("PDA_STG_PROJECT_REFERENCE_OS_PORTFOLIO")
PDA_STG_PROJECT_REFERENCE_OS_PORTFOLIO_df = PDA_STG_PROJECT_REFERENCE_OS_PORTFOLIO.get_dataframe()

PDA_STG_PROJECT_REFERENCE_ENRICH = dataiku.Dataset("PDA_STG_PROJECT_REFERENCE_ENRICH")
PDA_STG_PROJECT_REFERENCE_ENRICH_df = PDA_STG_PROJECT_REFERENCE_ENRICH.get_dataframe()

### Reading Slipstream Snowflake tables for Overrides

In [0]:
# Read recipe inputs from snowflake
PDA_MASTER_OVERRIDE_LIST = dataiku.Dataset("PDA_SLIPSTREAM_MASTER_OVERRIDE_LIST")
PDA_MASTER_OVERRIDE_LIST_SF_df = PDA_MASTER_OVERRIDE_LIST.get_dataframe()

PDA_PROJECT_OVERRIDE_RULES = dataiku.Dataset("PDA_SLIPSTREAM_PROJECT_OVERRIDE_RULES")
PDA_PROJECT_OVERRIDE_RULES_SF_df = PDA_PROJECT_OVERRIDE_RULES.get_dataframe()

### Applying Override Rules to Override Data with OneSource data

In [0]:
# reading Snowflake tables for override table and override rules
rules_df = PDA_PROJECT_OVERRIDE_RULES_SF_df.copy()
target_df = PDA_STG_PROJECT_REFERENCE_OS_PORTFOLIO_df.copy()
override_df = PDA_MASTER_OVERRIDE_LIST_SF_df.copy()
#target_df

In [0]:
OS_PORTFOLIO_OVERRIDDEN_df = override_dataframe(rules_df, target_df, override_df)

### Merge OneSource override data with OneSource Enrich Data

In [0]:
# function to merge OneSource override data with OneSource Enrich Data
def merge_os_df(LEFT_DF, RIGHT_DF, key='CANDIDATE_CODE'):
    try:
        # Finding Common columns
        common_cols = list(np.intersect1d(LEFT_DF.columns, RIGHT_DF.columns))
        common_cols.remove(key)

        # for each common column in left df it fills na values from right df  and finally merges two dfs(needs to confirm)
        for col in common_cols:
            LEFT_DF[col].fillna(RIGHT_DF[col])
            RIGHT_DF = RIGHT_DF.drop(col,axis=1)
        DF = pd.merge(LEFT_DF,RIGHT_DF,on=key,  how='left')

        # sorts the columns alphabetically
        DF = DF.reindex(sorted(DF.columns), axis=1)

        #move the PROJECT_ID column to first of the dataframe
        starting_cols = ['PROJECT_ID','PORTFOLIO_ID',key]
        DF = DF[ starting_cols + [ col for col in DF.columns if col not in starting_cols] ]
        DF[['PROJECT_ID','PORTFOLIO_ID']] = DF[['PROJECT_ID','PORTFOLIO_ID']].astype(int)

        return DF
    except Exception as e:
        print('merge_os_df failed, error:', str(e))
        pass

In [0]:
# calling the function to merge data based on condition
LEFT_DF = PDA_STG_PROJECT_REFERENCE_ENRICH_df.copy()
RIGHT_DF = OS_PORTFOLIO_OVERRIDDEN_df.copy()

In [0]:
PDA_MV_PROJECT_METADATA_df = merge_os_df(LEFT_DF, RIGHT_DF)
# PDA_MV_PROJECT_METADATA_df

### Writing to output

In [0]:
# PDA_STG_MV_PROJECT_METADATA = dataiku.Dataset("PDA_STG_MV_PROJECT_METADATA")
# PDA_STG_MV_PROJECT_METADATA.write_with_schema(PDA_MV_PROJECT_METADATA_df)

# PDA_STG_MV_PROJECT_MASTER = dataiku.Dataset("PDA_STG_MV_PROJECT_MASTER")
# PDA_STG_MV_PROJECT_MASTER.write_with_schema(OS_PORTFOLIO_OVERRIDDEN_df)

# PORTFOLIO METADATA

### Reading Input Data

In [0]:
PDA_STG_ENRICH_PORTFOLIO_METADATA = dataiku.Dataset("PDA_STG_ENRICH_PORTFOLIO_METADATA")
PDA_STG_ENRICH_PORTFOLIO_METADATA_df = PDA_STG_ENRICH_PORTFOLIO_METADATA.get_dataframe()

PDA_SLIPSTREAM_PORTFOLIO_ATTRIBUTES = dataiku.Dataset("PDA_SLIPSTREAM_PORTFOLIO_ATTRIBUTES")
PDA_SLIPSTREAM_PORTFOLIO_ATTRIBUTES_df = PDA_SLIPSTREAM_PORTFOLIO_ATTRIBUTES.get_dataframe()

### Data Processing

In [0]:
# merging data
PDA_STG_ENRICH_PORTFOLIO_METADATA_MERGED_df = PDA_STG_ENRICH_PORTFOLIO_METADATA_df.merge(PDA_SLIPSTREAM_PORTFOLIO_ATTRIBUTES_df,
             how='left', on=["PORTFOLIO_ID"] ,suffixes=('_DROPME', '')).filter(regex='^(?!.*_DROPME)')

### Write recipe outputs

In [0]:
# deleteme_Portfolio_Metadata = dataiku.Dataset("PDA_TEST_MV_PORTFOLIO_METADATA")
# deleteme_Portfolio_Metadata.write_with_schema(PDA_STG_ENRICH_PORTFOLIO_METADATA_MERGED_df)

# OUTPUT TIMESERIES

### Reading Input Data

In [0]:
PDA_OUTPUT_TIMESERIES_TYPES_df = dataiku.Dataset("PDA_SLIPSTREAM_TIMESERIES_TYPES")
PDA_OUTPUT_TIMESERIES_TYPES_df = PDA_OUTPUT_TIMESERIES_TYPES_df.get_dataframe()

#Replacing OS Global and Regional PNLs with Unioned snowflake PNL (pre-filtered by Portfolio ID)
PDA_STG_ENRICH_PNL = dataiku.Dataset("PDA_STG_ENRICH_PNL")
PDA_STG_ENRICH_PNL_df = PDA_STG_ENRICH_PNL.get_dataframe()

#Adding Dev Costs - this is a bit of a force fit may need to run as a separate table, depending on how captario structures
PDA_STG_ENRICH_DC = dataiku.Dataset("PDA_STG_ENRICH_DC")
PDA_STG_ENRICH_DC_df = PDA_STG_ENRICH_DC.get_dataframe()

### Data Processing

In [0]:
#Sharepoint Derived listing to edit see:
# https://pfizer.sharepoint.com/sites/PDADATASETS/CORE/Lists/PDA_OUTPUT_TIMESERIES_TYPES
#list includes both PNL and Dev Cost elements, Implicit assumption there is no naming overlap between these entities
SYSTEM_TIMESERIES_list = PDA_OUTPUT_TIMESERIES_TYPES_df['SYSTEM_TIMESERIES'].tolist()
SYSTEM_TIMESERIES_list = list(set(SYSTEM_TIMESERIES_list))

#creating specific list relevant to PNL, unpivot operation will fail if given invalid column references
SYSTEM_TIMESERIES_PNL_list =  list(set(SYSTEM_TIMESERIES_list) & set(PDA_STG_ENRICH_PNL_df.columns.values.tolist()))


# Unpivot PNL Data
PDA_STG_ENRICH_PNL_FILTERED_NORMALIZED_df = pd.melt(PDA_STG_ENRICH_PNL_df,
                                             id_vars=['PORTFOLIO_ID', 'PROJECT_ID', 'SNAPSHOT_ID','CANDIDATE_CODE',
                                                      'SOURCE','COMPONENT', 'REGION','YEAR','SCENARIO','PHASE'],
                                             value_vars= SYSTEM_TIMESERIES_PNL_list,
                                              var_name='TYPE', value_name='VALUE')

#Union PNL And Dev costs
PDA_STG_CONCAT_PNL_DC_df = pd.concat([PDA_STG_ENRICH_PNL_FILTERED_NORMALIZED_df,PDA_STG_ENRICH_DC_df])

### Write recipe outputs

In [0]:
# PDA_MV_OUTPUT_TIMESERIES = dataiku.Dataset("PDA_STG_MV_OUTPUT_TIMESERIES")
# PDA_MV_OUTPUT_TIMESERIES.write_with_schema(PDA_STG_CONCAT_PNL_DC_df)

# OUTPUT VALUES

### Reading Input Data

In [0]:
PDA_SLIPSTREAM_PORTFOLIO_ATTRIBUTES = dataiku.Dataset("PDA_SLIPSTREAM_PORTFOLIO_ATTRIBUTES")
PDA_SLIPSTREAM_PORTFOLIO_ATTRIBUTES_df = PDA_SLIPSTREAM_PORTFOLIO_ATTRIBUTES.get_dataframe()

PDA_SLIPSTREAM_VALUE_TYPES = dataiku.Dataset("PDA_SLIPSTREAM_VALUE_TYPES")
PDA_SLIPSTREAM_VALUE_TYPES_df = PDA_SLIPSTREAM_VALUE_TYPES.get_dataframe()

PDA_STG_ENRICH_FLATFILE = dataiku.Dataset("PDA_STG_ENRICH_FLATFILE")
PDA_STG_ENRICH_FLATFILE_df = PDA_STG_ENRICH_FLATFILE.get_dataframe()

PDA_STG_ENRICH_DC = dataiku.Dataset("PDA_STG_ENRICH_DC")
PDA_STG_ENRICH_DC_df = PDA_STG_ENRICH_DC.get_dataframe()

### Data Processing

In [0]:
FF_ECONOMICS_df =  pd.merge( PDA_SLIPSTREAM_VALUE_TYPES_df, PDA_STG_ENRICH_FLATFILE_df, how='inner', left_on='SYSTEM_VALUES', right_on='COLUMN_NAME')[['PORTFOLIO_ID','PROJECT_ID','SNAPSHOT_ID','CANDIDATE_CODE','COLUMN_NAME', 'COLUMN_DATA']]
FF_ECONOMICS_df.rename(columns={'COLUMN_NAME': 'TYPE', 'COLUMN_DATA': 'VALUE'}, inplace=True)

### Write recipe outputs

In [0]:
# PDA_TMP_MV_OUTPUT_VALUES = dataiku.Dataset("PDA_STG_MV_OUTPUT_VALUES")
# PDA_TMP_MV_OUTPUT_VALUES.write_with_schema(FF_ECONOMICS_df)